# Jupyter Notebook to clean install all the contracts

# Set Up Environment

In [18]:
network = "yeouido"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'yeouido_DevTestcontracts_20201210064637.pkl'
#contracts_20201109120425

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [19]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value
            

In [20]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




### Required service

In [21]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

# Importing wallet

### Importing from private key and printing ICX balance

In [42]:
private1="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet1 = KeyWallet.load(bytes.fromhex(private1))
print(deployer_wallet1.get_address())

private2="94a1471d2fa8114f585bb0c7baca343b2d57d2c65b2d9c0f461da828e3741f04"
deployer_wallet2 = KeyWallet.load(bytes.fromhex(private2))
print(deployer_wallet2.get_address())

private3="d7776d1ae9103bd49eeed8463e4ce50480b1f4bc760108325415998c239fcee5"
deployer_wallet3 = KeyWallet.load(bytes.fromhex(private3))
print(deployer_wallet3.get_address())

hx91bf040426f226b3bfcd2f0b5967bbb0320525ce
hx8c8ccc79c99375de5995ca6057cda7c334d9dd35
hx49c8a4c31e84b9abe7f8c8395eb7133b2151aeae


### Function to switch wallets to run transaction

In [23]:
wallets={'1':deployer_wallet1,'2':deployer_wallet2,'3':deployer_wallet3}
def wallet_selector():
    wallet=input("Enter the wallet from which you want to run the transaction")
    if str(wallet) not in wallets.keys():
        print("invalid wallet selected")
        return null
    return wallets[str(wallet)]


In [24]:
#wallet=wallet_selector()
#print(wallet.get_address())
pprint(contracts)

{'addressProvider': 'cx6b8f62c32f848491f9656635df7d13da1fade9d0',
 'feeProvider': 'cx31fbe153d6d264a5e79e1b60925f4ababe27806a',
 'lendingPool': 'cx7976fd32bcb0da0dd8b36f70c46781910ae108ee',
 'lendingPoolCore': 'cx788ac70e7c0e44b64635ec2c9c6198ef1847f156',
 'lendingPoolDataProvider': 'cx821b32a558db1e7cc264c9b5e9c21f946b05734f',
 'liquidationManager': 'cx5e050e62c2b7cde4cde02466c6d0c4e1285a90b4',
 'oICX': 'cx8c4af6ca127778a83731f2a2fd8de3bc2dd91ddf',
 'oToken': 'cx9fa4a72a27f62cb5619d392612c888462a0b7d3e',
 'priceOracle': 'cx6556d2d98059d59c99b982f6a7831f2b7bdc46f8',
 'sample_token': 'cx47cc26110b7346d0038765908ca363948c478f48',
 'sicx': 'cx2ec4b2b183c5ff417c55f64bf67374a8061d83e5'}


# Depositing USDb from a wallet

In [10]:
deployer_wallet=wallet_selector()
data = "{\"method\": \"deposit\", \"params\": {\"amount\": 1000000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool'], "_value": 1000000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xc4797ace918f983b1024f95d4f8cc8bf4972cfc5df96d2387f024e4254240606',
 'blockHeight': 10142381,
 'blockHash': '0x91fb2d6d920a5bd9ccac0943662e603c6bf9af211524559e1658e0518221a863',
 'txIndex': 1,
 'to': 'cx19be1be7b4e7750863edf966faac3f42dec21025',
 'stepUsed': 834040,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 834040,
 'eventLogs': [{'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x3635c9adc5dea00000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['PrintData(str,int,int,int)',
    'rate value core line 412',
    '0x763c82d21c3a',
    '0x26ab95c0320bff'],
   'data': ['0x0']},
  {'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx19be1be7b4e7750863edf966faac3f42dec21025',
    '0x763c82d21c3a',
    '0x26ab95c0320bff'],
   'data': ['0

# Borrowing USDb

In [75]:
deployer_wallet=wallet_selector()
params ={"_reserve": contracts['sample_token'], "_amount":150*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx9a1e70f60ee138d8e2c2eee0daf28a7d0645bd02', 'stepLimit': '0x11a350', 'timestamp': '0x5b3ab99c871a5', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'borrow', 'params': {'_reserve': 'cx2c002610b8aeaff84f6cb97c28294b1986016143', '_amount': '0x821ab0d4414980000'}}, 'signature': 'nnBAhBv6BGeWZE63Rr6d02x+c/R36yjf49WvR/G5VLd6hz7tKN6lL6lG/kghrjVg0ZPWIf2KdJi+rmBPoRrFWAA='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x4f24f998ab722bbcdd6589d23f1bd67cc98a4f30ff863710b291f60545017e0f',
 'blockHeight': 443982,
 'blockHash': '0x795f4716b2053eca2b1b8585c02c10fe30705843841760035bf91b80abb581c9',
 'txIndex': 1,
 'to': 'cx9a1e70f60ee138d8e2c2eee0daf28a7d0645bd02',
 'stepUsed': 1055920,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1055920,
 'eventLogs': [{'scoreAddress': 'cx879e276135a6f25236a8abf2e4c9801887538f60',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx2c002610b8aeaff84f6cb97c28294b1986016143',
    '0x864ae65482800',
    '0x3e2c284391c000'],
   'data': ['0xde0b6b3a7640000', '0xde0b6b3a7640000']},
  {'scoreAddress': 'cx2c002610b8aeaff84f6cb97c28294b1986016143',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cx879e276135a6f25236a8abf2e4c9801887538f60',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce',
    '0x821ab0d4414980000'],
   'data': ['0x4e6f6e65']}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x0

# Reedeming/Withdrawing USDb

In [ ]:
deployer_wallet=wallet_selector()
params ={"_amount": 10*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken']) \
    .nid(NID) \
    .nonce(100) \
    .method("redeem")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Repaying USDb

In [ ]:
deployer_wallet=wallet_selector()
data = "{\"method\": \"repay\", \"params\": {\"amount\": 100000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool'], "_value": 100000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sicx'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Depositing ICX

In [13]:
deployer_wallet=wallet_selector()
params = {"_amount": 10 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['lendingPool']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .value(10 * 10 ** 18) \
    .method("deposit") \
    .params(params) \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x081e8527008f2f7250bd40b645e192bf61ca5bb9c348055e303fcc9e7a117907',
 'blockHeight': 10142461,
 'blockHash': '0xe606a4cd37963728a8c3caef2eb8881cd0018945e75c6b9d8ec07c0ae21eea85',
 'txIndex': 1,
 'to': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
 'stepUsed': 723640,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 723640,
 'eventLogs': [{'scoreAddress': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
    'cx3e9d38175d112cd5cf05a90080cce779f202ba27',
    '0x8ac7230489e80000'],
   'data': []},
  {'scoreAddress': 'cx3e9d38175d112cd5cf05a90080cce779f202ba27',
   'indexed': ['Mint(Address,int)',
    'cx327a386a9f8076045679fc132a8baa9e270ca43b',
    '0x8ac7230489e80000'],
   'data': []},
  {'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x8ac7230489e80000',
    '0x0'],
   'data': ['0

# Withdrawing ICX 

In [ ]:
deployer_wallet=wallet_selector()
params = {"_amount": 250 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address())\
    .to(contracts['oICX'])\
    .nid(3) \
    .step_limit(10000000) \
    .nonce(100) \
    .method("redeem") \
    .params(params) \
    .build()
signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Borrow sICX

In [ ]:
deployer_wallet=wallet_selector()
params ={"_reserve": contracts['sicx'], "_amount":150 * 10 ** 18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Repay sICX

In [ ]:
deployer_wallet=wallet_selector()
data = "{\"method\": \"repay\", \"params\": {\"amount\": 100 * 10 ** 18}}".encode("utf-8")
params = {"_to": contracts['lendingPool'], "_value": 100 * 10 ** 18, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sicx'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

## Liquidataion Call

In [ ]:
#Specify the wallet of the borrower
_borrowerUser = wallet_selector()
#Specify the wallet with which you wish to repay _borrowUser's borrowed asset
_liquidatorUser = wallet_selector()
_loanPayoffAmount = 
depositData = {'method': 'liquidationCall', 'params': 
            {'_collateral': contracts['sicx'],
            '_reserve': contracts['sample_token'],
            '_user': _borrowerUser.get_address(),
            '_purchaseAmount': _loanPayoffAmount}}
data = json.dumps(depositData).encode('utf-8')
params = {"_to": contracts['lendingPool'],
            "_value": _loanPayoffAmount, "_data": data}
call_transaction = CallTransactionBuilder() \
    .from_(_liquidatorUser.get_address()) \
    .to(contracts['sample_token']) \
    .nid(3) \
    .step_limit(10000000) \
    .nonce(100) \
    .method("transfer") \
    .params(params) \
    .build()
signed_transaction = SignedTransaction(call_transaction, _liquidatorUser)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Readonly methods to get data

## Getting specific reserve data

### This returns the reserve data of USDb

In [25]:
params = {'_reserve': contracts['sample_token']}
_call = CallBuilder()\
    .from_(deployer_wallet1.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if values=="0x1" or values =="0x0" or "Address" in key:
        continue
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)

pprint(response)

{'availableLiquidity': 3500.0,
 'baseLTVasCollateral': 0.33,
 'borrowCumulativeIndex': 1.0,
 'borrowRate': 0.018333333333333333,
 'borrowingEnabled': '0x1',
 'decimals': 1.8e-17,
 'isActive': '0x1',
 'isFreezed': '0x0',
 'lastUpdateTimestamp': 1607490593422876,
 'liquidationBonus': 1e-17,
 'liquidationThreshold': 0.65,
 'liquidityCumulativeIndex': 1.0,
 'liquidityRate': 0.0020625,
 'oTokenAddress': 'cx9fa4a72a27f62cb5619d392612c888462a0b7d3e',
 'reserveAddress': 'cx47cc26110b7346d0038765908ca363948c478f48',
 'totalBorrows': 500.0,
 'totalLiquidity': 4000.0,
 'usageAsCollateralEnabled': '0x1'}


### This method returns the reserve data of sICX

In [26]:
params = {'_reserve': contracts['sicx']}
_call = CallBuilder()\
    .from_(deployer_wallet1.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if values=="0x1" or values =="0x0" or "Address" in key:
        continue
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)

pprint(response)

{'availableLiquidity': 5000.0,
 'baseLTVasCollateral': 0.33,
 'borrowCumulativeIndex': 1.0,
 'borrowRate': '0x0',
 'borrowingEnabled': '0x1',
 'decimals': 1.8e-17,
 'isActive': '0x1',
 'isFreezed': '0x0',
 'lastUpdateTimestamp': 1607489922569025,
 'liquidationBonus': 1e-17,
 'liquidationThreshold': 0.65,
 'liquidityCumulativeIndex': 1.0,
 'liquidityRate': '0x0',
 'oTokenAddress': 'cx8c4af6ca127778a83731f2a2fd8de3bc2dd91ddf',
 'reserveAddress': 'cx2ec4b2b183c5ff417c55f64bf67374a8061d83e5',
 'totalBorrows': '0x0',
 'totalLiquidity': 5000.0,
 'usageAsCollateralEnabled': '0x1'}


## Getting data of all reserves

### This returns the reserve data of all the reserves

In [28]:
params = {'_reserve': contracts['sample_token']}
_call = CallBuilder()\
    .from_(deployer_wallet1.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getAllReserveData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if v=="0x1" or v =="0x0" or "Address" in k:
            continue
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

{'USDb': {'availableLiquidity': 3500.0,
          'baseLTVasCollateral': 0.33,
          'borrowCumulativeIndex': 1.0,
          'borrowRate': 0.018333333333333333,
          'borrowingEnabled': '0x1',
          'decimals': 1.8e-17,
          'isActive': '0x1',
          'isFreezed': '0x0',
          'lastUpdateTimestamp': 1607490593422876,
          'liquidationBonus': 1e-17,
          'liquidationThreshold': 0.65,
          'liquidityCumulativeIndex': 1.0,
          'liquidityRate': 0.0020625,
          'oTokenAddress': 'cx9fa4a72a27f62cb5619d392612c888462a0b7d3e',
          'reserveAddress': 'cx47cc26110b7346d0038765908ca363948c478f48',
          'totalBorrows': 500.0,
          'totalLiquidity': 4000.0,
          'usageAsCollateralEnabled': '0x1'},
 'sICX': {'availableLiquidity': 5000.0,
          'baseLTVasCollateral': 0.33,
          'borrowCumulativeIndex': 1.0,
          'borrowRate': '0x0',
          'borrowingEnabled': '0x1',
          'decimals': 1.8e-17,
          'isActive

## Getting user account data

### Returns the user data of all the reserves combined

In [37]:
deployer_wallet=wallet_selector()
params = {'_user': deployer_wallet.get_address()}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getUserAccountData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'borrowingPower': 0.0,
 'currentLiquidationThreshold': 0.0,
 'currentLtv': 0.0,
 'healthFactor': -1e-18,
 'healthFactorBelowThreshold': 0.0,
 'totalBorrowBalanceUSD': 0.0,
 'totalCollateralBalanceUSD': 0.0,
 'totalFeesUSD': 0.0,
 'totalLiquidityBalanceUSD': 0.0}


### Getting user reserve data for USDb reserve

In [39]:
#deployer_wallet=wallet_selector()
params = {'_user': deployer_wallet.get_address() ,"_reserve":contracts['sample_token']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getUserReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'borrowRate': 0.018333333333333333,
 'currentBorrowBalance': 0.0,
 'currentBorrowBalanceUSD': 0.0,
 'currentOTokenBalance': 0.0,
 'currentOTokenBalanceUSD': 0.0,
 'lastUpdateTimestamp': 0,
 'liquidityRate': 0.0020625,
 'originationFee': 0.0,
 'principalBorrowBalance': 0.0,
 'principalBorrowBalanceUSD': 0.0,
 'useAsCollateral': 0.0,
 'userBorrowCumulativeIndex': 0.0}


### Getting user reserve data for sICX reserve

In [16]:
deployer_wallet=wallet_selector()
params = {'_user': deployer_wallet.get_address() ,"_reserve":contracts['sample_token']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getUserReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'borrowRate': 0.0,
 'currentBorrowBalance': 0.0,
 'currentBorrowBalanceUSD': 0.0,
 'currentOTokenBalance': 0.0,
 'currentOTokenBalanceUSD': 0.0,
 'lastUpdateTimestamp': 0,
 'liquidityRate': 0.0,
 'originationFee': 0.0,
 'principalBorrowBalance': 0.0,
 'principalBorrowBalanceUSD': 0.0,
 'useAsCollateral': 0.0,
 'userBorrowCumulativeIndex': 0.0}


### Getting the data of user for all reserves

In [37]:
deployer_wallet=wallet_selector()
params = {'_user': deployer_wallet.get_address()}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getUserAllReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

invalid wallet selected


NameError: name 'null' is not defined

## Get reserve configurations


### Getting configuration of USDb reserve

In [20]:
params = {"_reserve":contracts['sample_token']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getReserveConfigurationData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    response[key]=int(values,0)
pprint(response)


{'baseLTVasCollateral': 600000000000000000,
 'borrowingEnabled': 1,
 'decimals': 18,
 'isActive': 1,
 'liquidationBonus': 10,
 'liquidationThreshold': 650000000000000000,
 'usageAsCollateralEnabled': 1}


### Getting configuration of sICX reserve

In [17]:
params = {"_reserve":contracts['sicx']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getReserveConfigurationData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    response[key]=int(values,0)
pprint(response)

{'baseLTVasCollateral': 330000000000000000,
 'borrowingEnabled': 1,
 'decimals': 18,
 'isActive': 1,
 'liquidationBonus': 10,
 'liquidationThreshold': 650000000000000000,
 'usageAsCollateralEnabled': 1}


### Get all reserves configurations

In [40]:
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getAllReserveConfigurationData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        response[key][k]=int(v,0)
pprint(response)


{'USDb': {'baseLTVasCollateral': 330000000000000000,
          'borrowingEnabled': 1,
          'decimals': 18,
          'isActive': 1,
          'liquidationBonus': 10,
          'liquidationThreshold': 650000000000000000,
          'usageAsCollateralEnabled': 1},
 'sICX': {'baseLTVasCollateral': 330000000000000000,
          'borrowingEnabled': 1,
          'decimals': 18,
          'isActive': 1,
          'liquidationBonus': 10,
          'liquidationThreshold': 650000000000000000,
          'usageAsCollateralEnabled': 1}}


### Printing ICX balance from respective wallets

In [43]:
print('ICX balance of deployer_wallet3',icon_service.get_balance(deployer_wallet3.get_address()))
print('ICX balance of deployer_wallet1', icon_service.get_balance(deployer_wallet1.get_address()))
print('ICX balance of deployer_wallet2',icon_service.get_balance(deployer_wallet2.get_address()))  

ICX balance of deployer_wallet3 0
ICX balance of deployer_wallet1 8742417680630198414715
ICX balance of deployer_wallet2 0


## Realtime USDb balance of user(with accrued interest)

In [10]:
deployer_wallet=wallet_selector()
params = {'_owner': deployer_wallet.get_address() }
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken'])\
    .method("balanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)

print(int(response,0))

0


## Realtime ICX balance of user(with accured interest)

In [21]:
deployer_wallet=wallet_selector()
params = {'_owner': deployer_wallet.get_address() }
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oICX'])\
    .method("balanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)

print(int(response,0))


0


### Printing USDb balance from respective wallets

In [44]:
 wallets = [deployer_wallet1, deployer_wallet2, deployer_wallet3]
 for wallet in wallets:
    params = {'_owner': wallet.get_address()}
    _call = CallBuilder() \
        .from_(wallet.get_address()) \
        .to(contracts['sample_token']) \
        .method("balanceOf") \
        .params(params) \
        .build()
    response = icon_service.call(_call)
    print('USDb balance of ', wallet.get_address(), 'is' ,int(response,16))   

USDb balance of  hx91bf040426f226b3bfcd2f0b5967bbb0320525ce is 499994000000000000000000000
USDb balance of  hx8c8ccc79c99375de5995ca6057cda7c334d9dd35 is 0
USDb balance of  hx49c8a4c31e84b9abe7f8c8395eb7133b2151aeae is 0


### Printing sICX balance from respective wallets

In [24]:
 wallets = [deployer_wallet1, deployer_wallet2, deployer_wallet3]
 for wallet in wallets:
    params = {'_owner': wallet.get_address()}
    _call = CallBuilder() \
        .from_(wallet.get_address()) \
        .to(contracts['sicx']) \
        .method("balanceOf") \
        .params(params) \
        .build()
    response = icon_service.call(_call)
    print('sICX balance of ', wallet.get_address(), 'is' ,int(response,16))   

sICX balance of  hx91bf040426f226b3bfcd2f0b5967bbb0320525ce is 0
sICX balance of  hx8c8ccc79c99375de5995ca6057cda7c334d9dd35 is 0
sICX balance of  hx49c8a4c31e84b9abe7f8c8395eb7133b2151aeae is 0
